In [2]:
from google.cloud import bigquery
import os
import plotly.express as px
import pandas as pd


# Option A : Via variable d'environnement (Recommandé) 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../data/auth/sante-et-territoires-216daad01024.json" 
client = bigquery.Client() 
# Option B : Directement dans le code# 
#client = bigquery.Client.from_service_account_json("C:\\Users\stefl\Documents\Projet\sante-et-territoires-216daad01024.json")
#export GOOGLE_APPLICATION_CREDENTIALS="C:\Users\stefl\Documents\Projet\sante-et-territoires-216daad01024.json"

#client = bigquery.Client()

query = """
SELECT *
FROM `sante-et-territoires.sante.finess_occitanie_clean`
"""


df_communes = pd.read_csv("../data/geo/communes-france-2025.csv", sep=",", encoding="utf-8")



df = client.query(query).to_dataframe()




/tmp/ipykernel_14119/3219889902.py:22: DtypeWarning: Columns (1,9,11,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_communes = pd.read_csv("../data/geo/communes-france-2025.csv", sep=",", encoding="utf-8")


In [ ]:
#filtre sur les communes de l'occitanie
df_communes_occitanie = df_communes[df_communes['reg_nom'] == 'Occitanie']

In [ ]:
# latitudes longitudes en float
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)


In [ ]:
#création d'une carte des Hopitaux cliniques
import plotly.express as px

# 1. Filtrer sur un groupe
df_filtre = df[df['type d etablissements'] == "Hopitaux cliniques"]




# 2. Créer la carte (nouvelle API)
fig = px.scatter_map(
    df_filtre,
    lat="latitude",
    lon="longitude",
    hover_name="raison_sociale",
    hover_data={
        "libelle activite": True,
        "latitude": False,
        "longitude": False
    },
    color="type d etablissements",
    zoom=5,
    height=700
)

# 3. Style de carte (plus de mapbox_style)
fig.update_layout(
    map_style="open-street-map",   # nouvelle syntaxe
    margin={"r":0, "t":0, "l":0, "b":0}
)

fig.show()


In [32]:
#type d etablissements qui ont le champ soin renseigné
df_soin = df[df['libelle activite'].notna()]
print(df_soin['type d etablissements'].unique())

['Hopitaux cliniques' 'Soins de ville' 'Autres'
 'Coordination / Administration' "Centres d'accueil"]


In [33]:
df_soin = df[df['libelle activite'].notna()]

df_soin[['type d etablissements', 'libelle activite']]


,type d etablissements,libelle activite
1,Hopitaux cliniques,Traitement de l'insuffisance rénale chronique ...
2,Hopitaux cliniques,Traitement de l'insuffisance rénale chronique ...
3,Hopitaux cliniques,Traitement de l'insuffisance rénale chronique ...
4,Hopitaux cliniques,Traitement de l'insuffisance rénale chronique ...
36,Hopitaux cliniques,Médecine d'urgence
...,...,...
13511,Soins de ville,AMP DPN
13743,Hopitaux cliniques,Soins de longue durée
13744,Hopitaux cliniques,Soins de longue durée
13745,Hopitaux cliniques,Soins de longue durée


In [35]:
df_filtre = df[df['type d etablissements'] == "Hopitaux cliniques"]
df_filtre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2193 entries, 1 to 13746
Data columns (total 21 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   numero finess etablissement juridique   754 non-null    object 
 1   numero finess etablissement             2193 non-null   object 
 2   libelle description equipement sociaux  754 non-null    object 
 3   libta_equipement                        754 non-null    object 
 4   rsej                                    1013 non-null   object 
 5   activite                                1013 non-null   object 
 6   libelle activite                        1013 non-null   object 
 7   forme                                   1013 non-null   object 
 8   libelle forme                           1013 non-null   object 
 9   nofinessej_1                            2193 non-null   object 
 10  raison_sociale                          2193 non-null   object 


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13747 entries, 0 to 13746
Data columns (total 21 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   numero finess etablissement juridique   8035 non-null   object 
 1   numero finess etablissement             13747 non-null  object 
 2   libelle description equipement sociaux  8035 non-null   object 
 3   libta_equipement                        8035 non-null   object 
 4   rsej                                    1065 non-null   object 
 5   activite                                1065 non-null   object 
 6   libelle activite                        1065 non-null   object 
 7   forme                                   1065 non-null   object 
 8   libelle forme                           1065 non-null   object 
 9   nofinessej_1                            13747 non-null  object 
 10  raison_sociale                          13747 non-null  ob

In [37]:
# Je crée un dataframe pour les urgences uniquement 
df_urgences = df[df['libelle activite'].str.contains("urgence", case=False, na=False)]

In [38]:
df_urgences.head()

,numero finess etablissement juridique,numero finess etablissement,libelle description equipement sociaux,libta_equipement,rsej,activite,libelle activite,forme,libelle forme,nofinessej_1,...,rslongue,code commune,departement,libelle departement,categetab,categorie,latitude,longitude,type d etablissements,code_insee
36,None,90000183,None,None,CH ARIEGE COUSERANS,14.0,Médecine d'urgence,14.0,Non saisonnier,90781816,...,CENTRE HOSPITALIER ARIEGE COUSERANS SITE SAINT...,268,09,ARIEGE,355,Centre Hospitalier (C.H.),42.999011,1.128549,Hopitaux cliniques,09268
50,None,90000175,None,None,CHIVA,14.0,Médecine d'urgence,14.0,Non saisonnier,90781774,...,CHIVA SITE SAINT JEAN DE VERGES,264,09,ARIEGE,355,Centre Hospitalier (C.H.),43.028346,1.615951,Hopitaux cliniques,09264
52,None,90000175,None,None,CHIVA,14.0,Médecine d'urgence,0.0,Pas de forme,90781774,...,CHIVA SITE SAINT JEAN DE VERGES,264,09,ARIEGE,355,Centre Hospitalier (C.H.),43.028346,1.615951,Hopitaux cliniques,09264
556,None,110000023,None,None,CH CARCASSONNE,14.0,Médecine d'urgence,14.0,Non saisonnier,110780061,...,CENTRE HOSPITALIER CARCASSONNE ANTOINE GAYRAUD,69,11,AUDE,355,Centre Hospitalier (C.H.),43.219178,2.399894,Hopitaux cliniques,1169
559,None,110000023,None,None,CH CARCASSONNE,14.0,Médecine d'urgence,0.0,Pas de forme,110780061,...,CENTRE HOSPITALIER CARCASSONNE ANTOINE GAYRAUD,69,11,AUDE,355,Centre Hospitalier (C.H.),43.219178,2.399894,Hopitaux cliniques,1169


Fonction Haversine (distance en km)
🌍 Principe général
La Terre est (approximativement) une sphère.
Pour calculer la distance entre deux points sur une sphère, on ne peut pas utiliser la distance euclidienne classique.
La formule de Haversine permet de calculer la distance orthodromique, c’est‑à‑dire la distance la plus courte entre deux points à la surface de la Terre.


In [39]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # rayon de la Terre en km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [40]:
df_communes_occitanie.head()

,Unnamed: 0,code_insee,nom_standard,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,reg_code,reg_nom,dep_code,...,code_unite_urbaine,taille_unite_urbaine,population,superficie_hectare,superficie_km2,densite,latitude_centre,longitude_centre,grille_densite,grille_densite_texte
2813,2813,9001,Aigues-Juntes,aigues-juntes,AIGUES-JUNTES,COM,commune,76,Occitanie,9,...,9000,0.0,58,770,8,8.0,43.051,1.472,7,Rural à habitat très dispersé
2814,2814,9002,Aigues-Vives,aigues-vives,AIGUES-VIVES,COM,commune,76,Occitanie,9,...,9000,0.0,634,522,5,121.0,43.001,1.877,6,Rural à habitat dispersé
2815,2815,9003,L'Aiguillon,l-aiguillon,L'AIGUILLON,COM,commune,76,Occitanie,9,...,9000,0.0,379,646,6,59.0,42.910,1.900,6,Rural à habitat dispersé
2816,2816,9004,Albiès,albies,ALBIÈS,COM,commune,76,Occitanie,9,...,9000,0.0,121,779,8,16.0,42.754,1.698,6,Rural à habitat dispersé
2817,2817,9005,Aleu,aleu,ALEU,COM,commune,76,Occitanie,9,...,9000,0.0,126,1410,14,9.0,42.894,1.268,7,Rural à habitat très dispersé


In [41]:
# On crée une matrice de distances
distances = []

for idx, commune in df_communes_occitanie.iterrows():
    lat_c, lon_c = commune['latitude_centre'], commune['longitude_centre']
    
    # distances entre la commune et tous les établissements d'urgence
    d = haversine(
        lat_c, lon_c,
        df_urgences['latitude'].values,
        df_urgences['longitude'].values
    )
    
    # distance minimale = établissement le plus proche
    distances.append(d.min())

df_communes_occitanie['distance_urgence_km'] = distances


/tmp/ipykernel_14376/795381311.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
#distance moyenne à l'urgence en  occitanie
distance_moyenne = df_communes_occitanie['distance_urgence_km'].mean()
distance_moyenne


np.float64(17.33704782082205)

In [43]:
distance_par_dep = (
    df_communes_occitanie
    .groupby('dep_code')['distance_urgence_km']
    .mean()
    .reset_index()
    .sort_values('distance_urgence_km')
)


In [44]:
#csv des distances des communes aux urgences en occitanie
df_communes_occitanie.to_csv("../../data/processed/distances_communes_urgence_occitanie.csv", index=False)



In [45]:
df_communes_occitanie.head()

,Unnamed: 0,code_insee,nom_standard,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,reg_code,reg_nom,dep_code,...,taille_unite_urbaine,population,superficie_hectare,superficie_km2,densite,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,distance_urgence_km
2813,2813,9001,Aigues-Juntes,aigues-juntes,AIGUES-JUNTES,COM,commune,76,Occitanie,9,...,0.0,58,770,8,8.0,43.051,1.472,7,Rural à habitat très dispersé,11.967049
2814,2814,9002,Aigues-Vives,aigues-vives,AIGUES-VIVES,COM,commune,76,Occitanie,9,...,0.0,634,522,5,121.0,43.001,1.877,6,Rural à habitat dispersé,21.440885
2815,2815,9003,L'Aiguillon,l-aiguillon,L'AIGUILLON,COM,commune,76,Occitanie,9,...,0.0,379,646,6,59.0,42.910,1.900,6,Rural à habitat dispersé,23.506510
2816,2816,9004,Albiès,albies,ALBIÈS,COM,commune,76,Occitanie,9,...,0.0,121,779,8,16.0,42.754,1.698,6,Rural à habitat dispersé,31.229638
2817,2817,9005,Aleu,aleu,ALEU,COM,commune,76,Occitanie,9,...,0.0,126,1410,14,9.0,42.894,1.268,7,Rural à habitat très dispersé,16.284241


In [46]:
df_urgences.head()

,numero finess etablissement juridique,numero finess etablissement,libelle description equipement sociaux,libta_equipement,rsej,activite,libelle activite,forme,libelle forme,nofinessej_1,...,rslongue,code commune,departement,libelle departement,categetab,categorie,latitude,longitude,type d etablissements,code_insee
36,None,90000183,None,None,CH ARIEGE COUSERANS,14.0,Médecine d'urgence,14.0,Non saisonnier,90781816,...,CENTRE HOSPITALIER ARIEGE COUSERANS SITE SAINT...,268,09,ARIEGE,355,Centre Hospitalier (C.H.),42.999011,1.128549,Hopitaux cliniques,09268
50,None,90000175,None,None,CHIVA,14.0,Médecine d'urgence,14.0,Non saisonnier,90781774,...,CHIVA SITE SAINT JEAN DE VERGES,264,09,ARIEGE,355,Centre Hospitalier (C.H.),43.028346,1.615951,Hopitaux cliniques,09264
52,None,90000175,None,None,CHIVA,14.0,Médecine d'urgence,0.0,Pas de forme,90781774,...,CHIVA SITE SAINT JEAN DE VERGES,264,09,ARIEGE,355,Centre Hospitalier (C.H.),43.028346,1.615951,Hopitaux cliniques,09264
556,None,110000023,None,None,CH CARCASSONNE,14.0,Médecine d'urgence,14.0,Non saisonnier,110780061,...,CENTRE HOSPITALIER CARCASSONNE ANTOINE GAYRAUD,69,11,AUDE,355,Centre Hospitalier (C.H.),43.219178,2.399894,Hopitaux cliniques,1169
559,None,110000023,None,None,CH CARCASSONNE,14.0,Médecine d'urgence,0.0,Pas de forme,110780061,...,CENTRE HOSPITALIER CARCASSONNE ANTOINE GAYRAUD,69,11,AUDE,355,Centre Hospitalier (C.H.),43.219178,2.399894,Hopitaux cliniques,1169


In [ ]:
df_join = pd.read_csv("../../data/processed/finess_occitanie_join.csv")




# Calculs KPI
print(  'Personnes par établissement: ' + str(df_communes_occitanie["population"].sum()/df['numero finess etablissement'].nunique() ) )



Personnes par établissement: 601.9768092762895


In [82]:
df_cc = df_join[df_join['epci_nom'] == 'CC Pyrénées Audoises']

In [83]:
df_cc.head()

,numero finess etablissement juridique,numero finess etablissement,libelle description equipement sociaux,libta_equipement,rsej,activite,libelle activite,forme,libelle forme,nofinessej_1,...,code_unite_urbaine,taille_unite_urbaine,population,superficie_hectare,superficie_km2,densite,latitude_centre,longitude_centre,grille_densite,grille_densite_texte
535,NaN,110782455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110787017,...,11111.0,1.0,3066.0,2195.0,22.0,140.0,42.876,2.191,5.0,Bourgs ruraux
561,NaN,110007598,NaN,NaN,CH CARCASSONNE,14.0,Médecine d'urgence,14.0,Non saisonnier,110780061,...,11111.0,1.0,3066.0,2195.0,22.0,140.0,42.876,2.191,5.0,Bourgs ruraux
570,NaN,110780236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110780707,...,11111.0,1.0,3066.0,2195.0,22.0,140.0,42.876,2.191,5.0,Bourgs ruraux
590,NaN,110780194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330050899,...,11000.0,0.0,1144.0,1605.0,16.0,71.0,42.982,2.010,5.0,Bourgs ruraux
602,NaN,110007374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110786324,...,11111.0,1.0,3066.0,2195.0,22.0,140.0,42.876,2.191,5.0,Bourgs ruraux


In [92]:
df_cc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 535 to 1274
Data columns (total 48 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   numero finess etablissement juridique   23 non-null     float64
 1   numero finess etablissement             39 non-null     int64  
 2   libelle description equipement sociaux  23 non-null     object 
 3   libta_equipement                        23 non-null     object 
 4   rsej                                    1 non-null      object 
 5   activite                                1 non-null      float64
 6   libelle activite                        1 non-null      object 
 7   forme                                   1 non-null      float64
 8   libelle forme                           1 non-null      object 
 9   nofinessej_1                            39 non-null     int64  
 10  raison_sociale                          39 non-null     object 
 

In [88]:
df_communes_met_cc = df_communes_occitanie[df_communes_occitanie['epci_code'] == '200043776']

In [91]:
df_communes_met_cc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 3586 to 3985
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     61 non-null     int64  
 1   code_insee                     61 non-null     object 
 2   nom_standard                   61 non-null     object 
 3   nom_sans_accent                61 non-null     object 
 4   nom_standard_majuscule         61 non-null     object 
 5   typecom                        61 non-null     object 
 6   typecom_texte                  61 non-null     object 
 7   reg_code                       61 non-null     int64  
 8   reg_nom                        61 non-null     object 
 9   dep_code                       61 non-null     object 
 10  dep_nom                        61 non-null     object 
 11  canton_code                    61 non-null     object 
 12  canton_nom                     61 non-null     objec